In [58]:
from datetime import datetime, timedelta

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')

In [64]:
stemmer.stem('читать')

'чита'

In [ ]:
with open('data/football') as inp:
    print(next(inp))
    i = 0
    wordbags = []
    for line in inp:
        i += 1
        words = line.split('\t')[0].split()
        dttm = datetime.strptime(line[:-1].split('\t')[1], '%Y-%m-%d %H:%M:%S')
        if i > 10000000:
            break
        bag = []
        for word in words:
            sword = stemmer.stem(word.lower())
            if len(sword) > 2:
                bag.append(sword)
        wordbags.append((set(bag), dttm))

normal_query	datetime



In [76]:
len(wordbags)

0

In [46]:
football_related = []
for bag in wordbags:
    if 'футбол' in bag:
        football_related.append(bag)

In [47]:
len(football_related)

59144

In [48]:
related_dict = {}
for bag in football_related:
    for term in bag:
        related_dict[term] = related_dict.get(term, 0) + 1

In [49]:
for key, value in sorted(related_dict.items(), key=lambda item: item[1], reverse=True):
    print("%s: %s" % (key, value))

футбол: 59144
2018: 36167
чемпионат: 24348
мир: 24253
расписан: 7886
росс: 7126
таблиц: 6874
турнирн: 5814
матч: 5614
сборн: 3601
онлайн: 2781
прям: 2758
сегодн: 2609
трансляц: 2551
смотрет: 1866
игр: 1807
как: 1440
кто: 1351
игра: 1327
соста: 1318
эфир: 1298
испан: 1164
результат: 1095
хорват: 1090
скольк: 1080
фина: 1001
счет: 846
билет: 829
египет: 673
когд: 652
франц: 625
сейчас: 603
уругва: 567
где: 556
команд: 515
год: 504
групп: 502
москв: 491
будет: 488
куп: 488
про: 473
чемпион: 451
англ: 451
кана: 445
бельг: 415
июл: 415
дат: 414
арав: 392
город: 387
португал: 382
куличк: 382
саудовск: 381
июн: 379
бразил: 330
сетк: 323
герман: 319
песн: 319
что: 309
врем: 304
новост: 299
бесплатн: 298
канал: 287
открыт: 282
европ: 278
аргентин: 276
пле: 270
перв: 266
офф: 262
календар: 261
выигра: 256
ставк: 253
мест: 248
официальн: 242
кем: 230
сайт: 225
нижн: 224
тренер: 223
проведен: 219
рубл: 219
новгород: 215
проход: 214
все: 210
виде: 208
швец: 208
список: 208
2014: 201
гол: 201
самар:

подарок: 1
дьюл: 1
фирмен: 1
150a: 1
каналл: 1
legendaloko: 1
нотац: 1
лишн: 1
сможет: 1
манер: 1
иностранец: 1
растишк: 1
резултат: 1
челпион: 1
компьютерн: 1
изготовител: 1
avon: 1
чпм: 1
списык: 1
польщіжитт: 1
пентак: 1
леш: 1
инстаграмм: 1
34717: 1
dm6: 1
разврат: 1
польщ: 1
чемпионыммир: 1
канашин: 1
потраха: 1
flac: 1
версс: 1
лучшик: 1
ход: 1
аутсайдер: 1
новичк: 1
кумерта: 1
уменьша: 1
обесцен: 1
комсомольск: 1
автономн: 1
форебет: 1
блондинъъъэъъъъ: 1
волгограл: 1
3840х1080: 1
платн: 1
супермвркет: 1
синешалл: 1
сокольник: 1
планир: 1
арабиста: 1
велосипедист: 1
puppet: 1
жахон: 1
00звезд: 1
военнослужа: 1
пор: 1
дтп: 1
завернут: 1
именит: 1
8х8: 1
coub: 1
метк: 1
смел: 1
мтра: 1
боб: 1
революц: 1
khorvatiia: 1
1962: 1
торб: 1
помоч: 1
сповышенно: 1
отрывок: 1
обсира: 1
сопк: 1
жоз: 1
умира: 1
стереотип: 1
strana: 1
виск: 1
мухамед: 1
асхаб: 1
рисунк: 1
ролтон: 1
млгут: 1
горизонт: 1
томашевск: 1
rjvvtynfnjhs: 1
голеодор: 1
адриатик: 1
транслятор: 1
бьет: 1
снимет: 1
геро: 1


ремонт: 1
слова363: 1
щпбвдил: 1
выигоа: 1
мишкин: 1
вооружен: 1
приштин: 1
постро: 1
введ: 1
yuriy: 1
vkontakte: 1
белтг: 1
конч: 1
хорва: 1
фейсбук: 1
цитат: 1
дробышевск: 1
камчаск: 1
позицион: 1
скрап: 1
библилтечн: 1
мамикс: 1
коляск: 1
краснрярск: 1
достава: 1
расписанс: 1
бет: 1
стрелецк: 1
гонк: 1
расписание2018: 1
жадност: 1
интерва: 1
примьер: 1
экстрен: 1
mobb: 1
ава: 1
мсекир: 1
народ: 1
чон: 1
угон: 1
лиценз: 1
федоренк: 1
pdt: 1
гум: 1
понрав: 1
альман: 1
трудн: 1
фламенг: 1
калинкин: 1
зомбирован: 1
20игра: 1
просмотров1: 1
прл: 1
angrybirds: 1
студиорекомендуем: 1
вам6: 1
защит: 1
грошич: 1
достич: 1
смол: 1
насчет: 1
крымск: 1
fake: 1
вещан: 1
датчик: 1
51успех: 1
2018год: 1
иваныч: 1
сегоддн: 1
gucci: 1
увелич: 1
новокузнецк: 1
html: 1
сульм: 1
казин: 1
саундтрэк: 1
россиийск: 1
аугсбург: 1
схемс: 1
объясня: 1
ньютон: 1
мусор: 1
кзтз: 1
55путин: 1
чмпо: 1
main: 1
директор: 1
best: 1
китос: 1
кров: 1
обе: 1
нечина: 1
посуточн: 1
tarasov: 1
заполня: 1
виртуальн: 1
позав

In [62]:
min_dt, max_dt = datetime.now(), datetime.now() - timedelta(days=1800)
with open('data/football') as inp:
    print(next(inp))
    for line in inp:
        d = datetime.strptime(line[:-1].split('\t')[1], '%Y-%m-%d %H:%M:%S')
        if d < min_dt:
            min_dt = d
        if d > max_dt:
            max_dt = d
print(min_dt, max_dt)

normal_query	datetime

2018-06-14 00:00:00 2018-07-15 23:59:59
